<strong>Date :</strong> Créé le 03 Avril 2021| Mis à jour le 07 Avril 2021 </strong>

<strong>Compétition Kaggle - Team Théo
    
@auteur : </strong>Théo VEDIS

<strong>(2-4)_features_network2
      
Description :</strong> Le but de ce Notebook est de créer un second réseau à partir de nos données Reddit et d'en déduire des features qui seront utilisées dans notre modèle de Machine Learning. Ce réseau aura pour noeuds les auteurs de commentaires. Si un auteur répond à un commentaire d'un autre auteur, alors cette relation est matérialisée par une arête (dirigée dans le sens auteur qui répond vers l'autre auteur). 

Temps d'exécution du Notebook : environ  20min.

# Importation des librairies

In [1]:
# Librairies générales
import pandas as pd
import numpy as np
from tqdm import tqdm 
import json

# Librairie pour les réseaux 
import networkx as nx

# Chemin 

In [2]:
# Chemin relatif vers le dossier "data" (inutile de le changer).
pathFile = "../data/" 

# Chargement des données d'entrée



In [3]:
# On récupère le fichier créé dans le précédent Notebook. 
with open( pathFile + 'df_features_network1.json') as json_data:
    data_dict = json.load(json_data)

# On transforme le dictionnaire en DataFrame 
df = pd.DataFrame(data_dict)

# On libère de la RAM en supprimant la variable data_dict qui ne nous servira plus
del data_dict

In [4]:
df.head()

,created_utc,ups,link_id,name,author,parent_id,is_parent_comment,is_author_deleted,is_body_deleted,bot_comment,length_comment_chars_before_NLP,length_comment_chars_after_NLP,length_comment_words,nb_stopwords,sentiment,sentiment_child,depth,nb_direct_resp,nb_total_resp
0,1430438400,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,t1_cqug2sr,1,0,0,0,119,59,8,14,neg,0,2,0,0
1,1430438400,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,t3_34fvry,0,0,0,0,48,13,2,7,neg,0,2,0,0
2,1430438400,5.0,t3_34ffo5,t1_cqug90z,InterimFatGuy,t1_cqu80zb,1,0,0,0,4,4,1,0,neu,0,2,0,0
3,1430438401,1.0,t3_34aqsn,t1_cqug91c,JuanTutrego,t1_cqtdj4m,1,0,0,0,54,15,3,12,neg,0,2,0,0
4,1430438401,101.0,t3_34f9rh,t1_cqug91e,dcblackbelt,t1_cquc4rc,1,0,0,0,241,164,24,20,pos,0,2,1,2


# Création du réseau n°2
Pour rappel, ce réseau aura pour noeuds les auteurs de commentaires. Si un auteur répond à un commentaire d'un autre auteur, alors cette relation est matérialisée par une arête (dirigée dans le sens auteur qui répond vers l'autre auteur). 

In [5]:
# On commence par supprimer tous les auteurs "deleted" (en gardant une copie du dataframe complet)
df_copy = df.copy()
df.drop(index=df[df["author"] == "[deleted]"]["author"].index, axis=1, inplace=True)

In [6]:
len(set(df.author))

570734

Notre graphe sera constitué de 570,734 noeuds. 

In [7]:
# Création d'un réseau/graphe dirigé vide
g = nx.DiGraph()

## Ajout des noeuds 

In [8]:
g.add_nodes_from(df["author"], type="author")

## Ajout des arêtes 

L'idée ici est de considérer chaque paire (`author`, `parent_author`) et de créer une arête par paire. Pour cela, on crée d'abord un dictionnaire qui permet de mettre en relation un identifiant de commentaire `name` à un auteur. Ensuite, grâce à ce DataFrame, on constitue les paires (`author`, `parent_author`) pour chaque commentaire. 

In [9]:
# Dictionnaire commentaires-auteurs
dict_name_author = dict([(id,aut) for id, aut in zip(df['name'], df['author'])])

# Liste qui contiendra les paires 'author', 'parent_author'.
authors = []
# On parcourt le DataFrame en récupérant l'auteur du commentaire et son parent
for author, parent in zip(tqdm(df["author"]), df["parent_id"]):
    
    try : 
        # Grâce au dictionnaire on transforme l'identifiant du parent en son auteur
        parent_author = dict_name_author[parent]
        authors.append((author, parent_author))
    except : 
        pass

100%|██████████| 3922963/3922963 [00:04<00:00, 830048.53it/s]


In [10]:
len(set(authors))

2015831

Notre réseau sera constitué de plus de 2 millions de relations. 

In [11]:
# Ajout des arêtes. 
g.add_edges_from(authors, link_type="parent")

## Caractéristiques du graphe

In [12]:
print("Nombre de noeuds : ", len(g.nodes))
print("Nombre d'arêtes : ", len(g.edges))

Nombre de noeuds :  570734
Nombre d'arêtes :  2015831


Tout s'est bien passé puisqu'il y a bel et bien 570 734 noeuds et 2 015 831 arêtes dans le réseau (comme ce qui avait été anoncé précédemment).

# Création de features 

## (1) Centralité d'un noeud 
Dans un réseau (normalement plutôt non-dirigé mais fonctionne sur les réseau dirigé également), la centralité d'un noeud est liée à son implication avec d'autres acteurs, sa participation dans de nombreux liens. Peu importe si la proéminence est due à la réception ou à la transmission de nombreux liens.

Il existe plusieurs métriques permettant de mesurer la centralité, mais pour toutes l'objectif est de mesurer à quel point un noeud est impliqué dans le réseau, à quel point il est central. Intuitivement, plus l'acteur est connecté plus la centralité sera importante. 

Dans notre projet, <strong> plus la centralité de l'auteur d'un commentaire sera importante, plus son score a de bonnes chances d'être élevé</strong> car l'auteur étant au centre du réseau, son commentaire aura une plus forte visibilité qu'un commentaire ayant un auteur en périphérie.

### 1-1 Degré de centralité 
La première métrique est calculée à partir du degré d'un auteur. On ne distingue pas les degrés entrants (commentaires reçus pour un auteur) ou sortants (commentaires émis par un auteur). 
Interprétation :
- Valeur élevée : contact direct avec de nombreux autres auteurs
- Valeur faible : pas actif, auteur périphérique 

Remarque : Ce sera la centralité degré normalisée qui sera calculée. 

In [13]:
# Dictionnaire associant à chaque noeud leur centralité degré
degrees = nx.centrality.degree_centrality(g)

In [14]:
# A partir du dictionnaire précédent, on associe à chaque commentaire, la
# centralité degré de leur auteur.
df_copy['author_centrality_degree'] = [degrees[author]
                                       if author in degrees else 0 for author in tqdm(df_copy['author'])]

100%|██████████| 4234970/4234970 [00:03<00:00, 1317395.78it/s]


In [15]:
# Résultat 
df_copy['author_centrality_degree'][:5]

0    0.000138
1    0.000005
2    0.000126
3    0.000000
4    0.000004
Name: author_centrality_degree, dtype: float64

### 1-2 Centralité liée aux vecteurs propres
Pour cette métrique, l'idée est que l'importance d'un noeud dépend de l'importance de ses voisins (définition récursive). 

Remarque : Bien que généralement les différentes métriques de centralité sont corrélées positivement (quand ce n'est pas le cas cela met en évidence des noeuds cractéristiques), nous avons fait le choix , à ce stade, d'en garder plusieurs. C'est lors de l'apprentissage du modèle prédictif et surtout son amélioration que nous supprimerons peut être quelqu'unes de ces métriques.

In [16]:
# Temps exécution : 40s 
# Dictionnaire associant à chaque noeud leur centralité "eigenvector"
eigenvectors = nx.eigenvector_centrality(g)

In [17]:
# A partir du dictionnaire précédent, on associe à chaque commentaire, la
# centralité eigenvector de leur auteur.
df_copy['author_centrality_ev'] = [eigenvectors[author]
                                   if author in eigenvectors else 0 for author in tqdm(df_copy['author'])]
# Résultat 
df_copy['author_centrality_ev'][:5]

100%|██████████| 4234970/4234970 [00:03<00:00, 1296850.53it/s]


0    1.312352e-03
1    2.167334e-19
2    1.712427e-03
3    2.167334e-19
4    5.092290e-05
Name: author_centrality_ev, dtype: float64

### 1-3 Intermédiarité et proximité 
Nous aurions aimé calculer ces deux autres métriques vues en cours. Cependant, le graphe étant tellement important (2millions de relations) qu'après une nuit d'exécution, l'algortihme n'était toujours pas fini (arrêté au bout de 18h). 
Nous n'aurons donc pas de features liées à la centralité intermédiaire ou proximité. 

In [18]:
# Temps exécution : > 18h 
# closeness = nx.centrality.closeness_centrality(g)
# betwenness = nx.centrality.betweenness_centrality(g)

## (2) Prestige d'un noeud 
Comme pour la centrlité, son objectif est de mesurer l'importance d'un noeud dans un réseau. Cependant, il est un peu plus subtile que cette dernière puisqu'il distingue les relations entrantes des relations sortantes (il se calcule uniquement sur les graphes dirigés donc). Un noeud prestigieux est un noeud souvent référencé (dans notre cas un auteur qui reçoit beaucoup de commentaires).  

Une fois de plus, plusieurs métriques sont possibles.

### 2-1 Prestige - degré 
Comme pour la centralité, la première métrique est liée au degré d'un noeud, sauf qu'ici on ne considère que les noeuds entrants. 

In [19]:
prestige_degrees = nx.in_degree_centrality(g)

In [20]:
df_copy['author_prestige_degree'] = [prestige_degrees[author] if author in prestige_degrees else 0 for author in tqdm(df_copy['author'])]
df_copy['author_prestige_degree'][:5]

100%|██████████| 4234970/4234970 [00:03<00:00, 1283012.81it/s]


0    0.000072
1    0.000000
2    0.000051
3    0.000000
4    0.000002
Name: author_prestige_degree, dtype: float64

### 2-2 Autre prestige 
Comme pour la centralité, les autres métriques prennent trop de temps à être calculesé. Nous nous contenterons uniquement du prestige degré. <br> 
De plus, la librairie 'networkw' n'a pas de fonction qui permet de calculer directement ces autres métriques, ainsi si nous le faisions à la main, les fonctions seraient encore moins optimisées et prendraient énormément de temps. 

# Sauvegarde 

In [21]:
df_copy.head()

,created_utc,ups,link_id,name,author,parent_id,is_parent_comment,is_author_deleted,is_body_deleted,bot_comment,length_comment_chars_before_NLP,length_comment_chars_after_NLP,length_comment_words,nb_stopwords,sentiment,sentiment_child,depth,nb_direct_resp,nb_total_resp,author_centrality_degree,author_centrality_ev,author_prestige_degree
0,1430438400,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,t1_cqug2sr,1,0,0,0,119,59,8,14,neg,0,2,0,0,0.000138,1.312352e-03,0.000072
1,1430438400,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,t3_34fvry,0,0,0,0,48,13,2,7,neg,0,2,0,0,0.000005,2.167334e-19,0.000000
2,1430438400,5.0,t3_34ffo5,t1_cqug90z,InterimFatGuy,t1_cqu80zb,1,0,0,0,4,4,1,0,neu,0,2,0,0,0.000126,1.712427e-03,0.000051
3,1430438401,1.0,t3_34aqsn,t1_cqug91c,JuanTutrego,t1_cqtdj4m,1,0,0,0,54,15,3,12,neg,0,2,0,0,0.000000,2.167334e-19,0.000000
4,1430438401,101.0,t3_34f9rh,t1_cqug91e,dcblackbelt,t1_cquc4rc,1,0,0,0,241,164,24,20,pos,0,2,1,2,0.000004,5.092290e-05,0.000002


In [22]:
# Sauvegarde DataFrame dans un fichier 
df_copy.to_json(pathFile + "df_features_network2.json") 

# (3) Communautés 
A travers ce réseau, l'extraction de communautés semblait être pertinente. Nous pensions que détecter des groupes d'auteurs qui intéragissent souvent ensemble (et rarement avec d'autres auteurs ne faisant pas partie de leur communauté), pouvait être un indicateur permettant de prédire le score d'un commentaire.

Nous avons essayer une grande partie des algortihmes implémentés dans la librairie NetworkX, des algortihmes vus en cours (Clique Percolation), manipuler en TP (Girvan Newman, bipartition de Kernighan–Lin, Fluid communities algorithm), ou même de nouveaux algorithmes (Naive Greddy Modularity, Label propagation, etc). Cependant, une fois de plus, à cause de l'importance du nombre de noeuds, une grande majorité des algortihmes mettaient trop de temps à s'exécuter. L'inconvénient des méthodes implémentées dans les libraires est que nous ne savions pas le temps d'exécution restant (au contraire de nos méthodes en utilisant le TQDM). Ainsi, après quelques heures d'exécution, nous interrompions l'exécution sans savoir si elle allait se terminer dans 5h ou 5min... 

In [23]:
# Pour la détection de communautés, il faut utiliser un graphe non-dirigé
g = g.to_undirected()

In [24]:
# ngm = list(nx.community._naive_greedy_modularity_communities(g))
# gn = list(nx.community.girvan_newman(g))
# fluid = list(nx.community.asyn_fluidc(g, 3))
# label_propa = list(nx.community.label_propagation_communities(g))

L'un des deux seuls algortihmes qui nous a donné des résultats était la bipartition de Kernighan–Lin. Comme son nom l'indique, cet algortihme permet de séparer le réseau en deux. Ici, étant donné que le réseau était consitué de 500 000 auteurs, le séparer seulement en deux nous semblait être un peu juste pour que cela nous aide à prédire le score d'un commentaire. De plus, les métriques évaluant la qualité de ce partitionament nous indiquaient que la répartition n'était pas bonne. Nous avons décidé de ne pas en faire une feature. 


In [25]:
KL = nx.community.kernighan_lin_bisection(g)
nx.community.quality.coverage(g, KL)

0.5008972716698323

L'autre algorithme  qui fonctionnait, était CPM (Clique Percolation Method).Cependant, nous n'avons pas créé une feature à partir de lui pour plusieurs raisons. Tout d'abord, comme c'est un algorithme qui permet de détecter des communautés qui se chevauchent (un auteur peut appartenir à plusieurs communautés ou à aucune), son résultat n'est pas une partition. Ainsi, les métriques permettant d'évaluer la qualité d'un partitionnement (modularité, couverture, performance) ne fonctionnent pas. Il était donc difficile de savoir si les communautés extraites étaient de bonne qualité ou pas, et surtout, il était impossible de mesure s'il était mieux de prendre k=3, k=4, k5, ou plus. <br> 
De plus, peu d'acteurs étaient finalement concernés par l'extraction de communautés via la méthode de percolation (la majorité n'appartenait à aucune clique). On avait donc peu d'auteurs présent dans un grand nombre de communautés qui étaient majoritairement composés que de quelques auteurs. Ainsi, nous estimions que cette information ne serait pas clairement pas utile pour prédire le score d'un commentaire, nous avons donc décider de ne pas en faire une feature. 

In [26]:
# 10 min
perco3 = nx.community.k_clique_communities(g, 3)
perco3 = [set(i) for i in list(perco3)] 


# Ensemble qui contiendra les auteurs appartenant à au moins une communauté
ens = set()

# 3 min
for i in perco3:
    ens = ens.union(i)

print("Nombre de communautés pour k=3 :", len(perco3))

print("Nombre d'auteurs présent dans une des %d communautés : %d" %(len(perco3), len(ens)))

Nombre de communautés pour k=3 : 25475
Nombre d'auteurs présent dans une des 25475 communautés : 73411


In [27]:
# 2 min
perco4 = nx.community.k_clique_communities(g, 4)
perco4 = [set(i) for i in list(perco4)] 

# Ensemble qui contiendra les auteurs appartenant à au moins une communauté
ens = set()

# 30s
for i in perco4:
    ens = ens.union(i)

print("Nombre de communautés pour k=4 :", len(perco4))

print("Nombre d'auteurs présent dans une des %d communautés : %d" %(len(perco4), len(ens)))

Nombre de communautés pour k=4 : 1404
Nombre d'auteurs présent dans une des 1404 communautés : 7497


On constate que si k est petit, le nombre de communauté explose et si k est trop grand, trop peu d'auteurs sont consernés par les communautés. Il est impossible d'établir une feature à partir de la percolation de cliques. 